In [ ]:
#main code taken from here https://github.com/bnsreenu/python_for_microscopists/blob/master/181_multivariate_timeseries_LSTM_GE.py

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error

In [ ]:
#loading dataset without proportions (neg/pos) variables 

df = pd.read_csv('AllScorePerDay', index_col='dates')
dfP = pd.read_csv('AllPropPword', index_col='dates')

In [ ]:
#modifying  pandemic terms dataset 

dfP = dfP.shift(1)
dfP = dfP.fillna(np.mean(dfP))
df['Pword'] = dfP['0']

In [ ]:
#creating dummy variables, both datasets had 'holiday ouliers' in the same dates, so no need for manual codification

df['holiday'] = 0
df['covid'] = 0

for index, score in enumerate(df['score']):
    if score >0.04:
        df['holiday'][index] = 1
    if score < -0.18:
        df['covid'][index] = 1
        

In [ ]:
# differentiating main data and substituting 

df['score'] = df['score'].diff()
df = df.fillna(np.mean(df['score'][:366]))

In [ ]:
train = df
test = df[359:]

In [ ]:
#Train arrays and hyperparameter setting

trainX = []
trainY = []
testX = []
testY = []
train  = np.array(train)
test = np.array(test)
n_future = 1
n_past = 7

In [ ]:
#creation of arrays: 7 values and one target value

for i in range(n_past, len(test)-n_future+1):
    testX.append(test[i-n_past:i,0:test.shape[1]])
    #testY.append(test[i+n_future-1:i+n_future,0])
for i in range(n_past, len(train)-n_future+1):
    trainX.append(train[i-n_past:i,0:train.shape[1]])
    trainY.append(train[i+n_future-1:i+n_future,0])
trainX, trainY = np.array(trainX), np.array(trainY)
testX = np.array(testX)
testX.shape

In [ ]:
#LSTM model creation

model = Sequential()
model.add(LSTM(64,activation='relu',input_shape=(trainX.shape[1],trainX.shape[2]),return_sequences=True))
model.add(LSTM(32,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))
model.compile(optimizer='adam',loss='mse')
model.summary()

In [ ]:
#sanity-check for the arrrays created

trainY[358] == df.iloc[365]

In [ ]:
#model is stochastic so weights are either saved or kernel restarted to prevent the model from 'learning'

Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)

In [ ]:
#dinamically, one-step-ahead polarity prediction

preds = []
index2 = 0

start = dt.now()
for i in range(359,359+len(testX)):
    model.fit(trainX[:i],trainY[:i], epochs=1,batch_size=64,validation_split=0.1,verbose=0)
    pred = model.predict(testX[index2].reshape(1,trainX.shape[1],trainX.shape[2]))
    pred = pred[0][0] + df['score'].iloc[index2+365]
    preds.append(pred)
    index2 +=1
    if i==359+(len(testX)-1):
        
        print( (dt.now() - start).seconds)
    

In [ ]:
#RMSE calcaulation 

a = np.sqrt(mean_squared_error(df['score'][366:], preds))
a